In [3]:
import json, requests
import pandas as pd
import numpy as np
import io
import random
import time
from sklearn.pipeline import make_pipeline

import os

from joblib import dump, load

import cml.data_v1 as cmldata

from random import sample
import warnings
warnings.filterwarnings('ignore')

**Get test data**

In [2]:
CONNECTION_NAME = "loki-datalake"
# ## Sample Usage to get pandas data frame
EXAMPLE_SQL_QUERY = "show databases"
# Alternate Sample Usage to provide different credentials as optional parameters
conn = cmldata.get_connection(
   CONNECTION_NAME, {"USERNAME": os.environ['PROJECT_OWNER'], "PASSWORD": "D83j8TNG#PS#jAQ"}
)

In [ ]:
#import requests
#r = requests.post('https://modelservice.ml-5fea65b7-abd.loki-cdp.a465-9q4k.cloudera.site/model', data='{"accessKey":"m9ylzh0tv2kkxz8hzfq1ptaj34ldigjq","request":{"param":"value"}}', headers={'Content-Type': 'application/json'})

In [4]:
kudu_input = pd.read_csv('query-impala-319.csv')

In [5]:
kudu_input.sort_values(['symbol','symbol_time'],ascending=False).head()

,symbol,symbol_time,open,high,low,close,volume,last_refreshed,information
777,ZION,2023-01-10 19:19:00,51.029999,51.029999,51.020000,51.020000,300,2023-01-11 21:08:59,Real Time
774,XOM,2023-01-10 20:00:00,109.599998,109.599998,109.599998,109.599998,318,2023-01-11 21:02:49,Real Time
770,XOM,2023-01-10 19:58:00,109.599998,109.599998,109.599998,109.599998,200,2023-01-11 21:02:49,Real Time
761,XOM,2023-01-10 19:55:00,109.660004,109.690002,109.660004,109.690002,273,2023-01-11 21:02:49,Real Time
759,XOM,2023-01-10 19:43:00,109.699997,109.699997,109.699997,109.699997,111,2023-01-11 21:02:49,Real Time


In [6]:
#kudu_input.symbol.unique()[:10]
kudu_input.groupby('symbol').size()

symbol
AAL     40
AAPL    66
ALK      3
AMZN    80
BAX      1
        ..
WBD     32
WDC      1
WYNN     3
XOM     18
ZION     1
Length: 90, dtype: int64

### Automate the dictionary build

In [7]:
cols = ['symbol','symbol_time','close', 'volume']

In [8]:
# Model - new model- 
model_url = 'https://modelservice.ml-5fea65b7-abd.loki-cdp.a465-9q4k.cloudera.site/model'

# make access key environment variable
accessKey = 'm9ylzh0tv2kkxz8hzfq1ptaj34ldigjq'

In [9]:
kudu_input.head()

,symbol,symbol_time,open,high,low,close,volume,last_refreshed,information
0,AAL,2023-01-10 19:10:00,15.2000,15.2000,15.2000,15.2000,895,2023-01-11 21:02:05,Real Time
1,AAL,2023-01-10 18:44:00,15.2000,15.2000,15.2000,15.2000,123,2023-01-11 21:02:05,Real Time
2,AAL,2023-01-10 18:28:00,15.1700,15.1700,15.1700,15.1700,322,2023-01-11 21:02:05,Real Time
3,AAL,2023-01-10 19:24:00,15.2200,15.2200,15.2200,15.2200,3611,2023-01-11 21:02:05,Real Time
4,AAL,2023-01-10 19:52:00,15.2091,15.2091,15.2091,15.2091,130,2023-01-11 21:02:05,Real Time


In [10]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in cols:
#kudu_input.columns:
    input_data_mult[col] = kudu_input[col].to_list()

In [11]:
#input_data_mult
input_data_mult.keys()

dict_keys(['symbol', 'symbol_time', 'close', 'volume'])

In [12]:
input_data_mult['symbol'][:10]

['AAL', 'AAL', 'AAL', 'AAL', 'AAL', 'AAL', 'AAL', 'AAL', 'AAL', 'AAL']

In [13]:
input_data_mult['close'][:10]

[15.199999809265137,
 15.199999809265137,
 15.170000076293944,
 15.220000267028809,
 15.209099769592283,
 15.220000267028809,
 15.199999809265137,
 15.220000267028809,
 15.210000038146973,
 15.210000038146973]

In [14]:
input_data_mult['symbol_time'][:10]

['2023-01-10 19:10:00',
 '2023-01-10 18:44:00',
 '2023-01-10 18:28:00',
 '2023-01-10 19:24:00',
 '2023-01-10 19:52:00',
 '2023-01-10 19:47:00',
 '2023-01-10 18:43:00',
 '2023-01-10 19:55:00',
 '2023-01-10 19:33:00',
 '2023-01-10 19:37:00']

input data frame is df

In [15]:

    
# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [16]:
request_dict

{'accessKey': 'm9ylzh0tv2kkxz8hzfq1ptaj34ldigjq',
 'request': {'symbol': ['AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
   'AAPL',
 

In [8]:
#request_dict
kudu_input

In [17]:
r9 = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [18]:
r9.json()

{'success': False,
 'ReplicaID': 'rf-direction-predict-3-5-d6587f7b9-hfz9r',
 'Size': 137,
 'StatusCode': 400}

In [35]:
r9.json()['response']

{'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:61120f1b-0e6a-483d-a73d-6594c1f7ae6f/ccde68fe-a73c-4d60-89e1-77fe84b97b92',
 'prediction': {'pred_time': ['2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00',
   '2023-01-10 20:00:00

request_dict.keys()

In [37]:
r9.json()['response']['prediction'].keys()

dict_keys(['pred_time', 'prediction', 'symbol'])

In [226]:
df.Name

0     NVR
1     TER
2     MET
3    CTSH
4    KLAC
5    CSGP
6     ETN
7     IEX
8    AAPL
9     DAL
Name: Name, dtype: object